In [1]:
# notebooks/05_RoBERTa.ipynb
# Importar las bibliotecas necesarias
import os
import logging
import sys
import pandas as pd

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.file_utils import read_excel_file, save_data_to_excel
from utils.advanced_ner_utils import (
    perform_ner_with_roberta,
    process_sentences_and_extract_entities,
)
from config import XLSX_DIRECTORY
from utils.ner_utils import cluster_entities
from transformers import AutoTokenizer, AutoModelForTokenClassification


# Carga de modelos y tokenizers
roberta_model_name = "PlanTL-GOB-ES/roberta-large-bne-capitel-ner"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForTokenClassification.from_pretrained(roberta_model_name)


def main():
    # Cargar los datos de las oraciones desde el archivo Excel
    logging.info("Cargando datos de oraciones desde Excel...")
    sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))
    # sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence_2.xlsx"))

    # Realizar NER con RoBERTa
    logging.info("Realizando NER con RoBERTa...")
    roberta_entities_df = process_sentences_and_extract_entities(
        sentence_data,
        roberta_tokenizer,
        roberta_model,
        perform_ner_with_roberta,
        "roberta",
    )

    # Convertir DataFrame de entidades a lista de diccionarios para el clustering
    roberta_entities_df = roberta_entities_df.to_dict("records")

    # Opcional: Agrupar entidades similares con K-means
    logging.info("Agrupando entidades similares...")
    roberta_entities_df = cluster_entities(roberta_entities_df, num_clusters=10)

    # Guardar los datos de entidades RoBERTa en un nuevo archivo Excel
    logging.info("Guardando datos de entidades RoBERTa en Excel...")
    save_data_to_excel(roberta_entities_df, XLSX_DIRECTORY, "roberta_entity.xlsx")


if __name__ == "__main__":
    main()

2024-01-21 17:41:07,931 - INFO - loading weights file pytorch_model.bin from cache at C:\Users\User\.cache\huggingface\hub\models--PlanTL-GOB-ES--roberta-large-bne-capitel-ner\snapshots\87b8cbf77bf17db1e79441f5cb0c1a1ab4bf40bf\pytorch_model.bin
2024-01-21 17:41:10,222 - INFO - All model checkpoint weights were used when initializing RobertaForTokenClassification.

2024-01-21 17:41:10,229 - INFO - All the weights of RobertaForTokenClassification were initialized from the model checkpoint at PlanTL-GOB-ES/roberta-large-bne-capitel-ner.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForTokenClassification for predictions without further training.
2024-01-21 17:41:10,244 - INFO - Cargando datos de oraciones desde Excel...
2024-01-21 17:41:11,715 - INFO - Realizando NER con RoBERTa...


Procesando: 0/2208 oraciones (0.00%)
Procesando: 220/2208 oraciones (9.96%)
Procesando: 440/2208 oraciones (19.93%)
Procesando: 660/2208 oraciones (29.89%)
Procesando: 880/2208 oraciones (39.86%)
Procesando: 1100/2208 oraciones (49.82%)
Procesando: 1320/2208 oraciones (59.78%)
Procesando: 1540/2208 oraciones (69.75%)
Procesando: 1760/2208 oraciones (79.71%)

Entidad 'Decreto de Urgencia N�� 017' no encontrada en el texto.
Contexto: efectos del COVID - 19 [ENTIDAD] mediante el Decreto de Urgencia
Oración: No obstante, lo anterior, mediante el Decreto de Urgencia Nº 017-2022 se han destinado 96.8 millones de soles para atender las demandas de las ollas comunes y enfrentar el incremento de los precios producto del contexto internacional y de los efectos del COVID-19.
Omitiendo la adición de la entidad 'Decreto de Urgencia N�� 017' ya que no se encontró en el texto.
Procesando: 1980/2208 oraciones (89.67%)
Procesando: 2200/2208 oraciones (99.64%)


2024-01-21 17:55:26,256 - INFO - Agrupando entidades similares...


Procesamiento completado.


d:\TFM_Project\PLN_Project\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
2024-01-21 17:55:26,729 - INFO - Guardando datos de entidades RoBERTa en Excel...
2024-01-21 17:55:27,102 - INFO - DataFrame guardado en d:\TFM_Project\PLN_Project\data\xlsx\roberta_entity.xlsx


In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

# Carga de modelos y tokenizers
roberta_model_name = "PlanTL-GOB-ES/roberta-large-bne-capitel-ner"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForTokenClassification.from_pretrained(roberta_model_name)

def rebuild_entity_from_subtokens_roberta(subtokens, tokenizer):
    entity = ""
    for i, token in enumerate(subtokens):
        decoded_token = tokenizer.decode([token], skip_special_tokens=True)
        if i == 0:
            entity = decoded_token
        elif decoded_token.startswith("Ġ"):
            entity += " " + decoded_token[1:]
        else:
            entity += decoded_token

    entity = entity.replace(" - ", "-").replace(" — ", "—").strip()
    print(f"Entidad reconstruida (RoBERTa): {entity}")
    return entity

def perform_ner_with_roberta(text, tokenizer, model):
    tokens = tokenizer.encode_plus(
        text, return_tensors="pt", truncation=True, max_length=512
    )

    with torch.no_grad():
        outputs = model(**tokens)

    entities = []
    current_entity_tokens = []

    for idx, pred in enumerate(outputs.logits[0]):
        label = model.config.id2label[pred.argmax().item()]
        token = tokens.input_ids[0][idx]
        token_str = tokenizer.decode([token], skip_special_tokens=True)
        print(f"Índice: {idx}, Token: {token}, Decoded: {token_str}, Label: {label}")

        if label.startswith("B-") or label.startswith("S-"):
            if current_entity_tokens:
                entity = rebuild_entity_from_subtokens_roberta(current_entity_tokens, tokenizer)
                entities.append(entity)
            current_entity_tokens = [token]
        elif label.startswith("I-") or label.startswith("E-"):
            current_entity_tokens.append(token)
        else:
            if current_entity_tokens:
                entity = rebuild_entity_from_subtokens_roberta(current_entity_tokens, tokenizer)
                entities.append(entity)
                current_entity_tokens = []

    if current_entity_tokens:
        entity = rebuild_entity_from_subtokens_roberta(current_entity_tokens, tokenizer)
        entities.append(entity)

    return entities


# Oración de entrada
input_sentence = "Hemos llegado con el Estado y sus servicios articulados a los extremos de la Amazonía A comunidades como Montetoni, en el Cusco o Pampa Entsa, en Amazonas, para acercar los servicios básicos y los programas sociales del Estado."
extracted_entities = perform_ner_with_roberta(input_sentence, roberta_tokenizer, roberta_model)

print("\nEntidades extraídas:")
for entity in extracted_entities:
    print(entity)


Índice: 0, Token: 0, Decoded: , Label: O
Índice: 1, Token: 7688, Decoded:  Hemos, Label: O
Índice: 2, Token: 3997, Decoded:  llegado, Label: O
Índice: 3, Token: 350, Decoded:  con, Label: O
Índice: 4, Token: 344, Decoded:  el, Label: O
Índice: 5, Token: 1614, Decoded:  Estado, Label: S-ORG
Índice: 6, Token: 342, Decoded:  y, Label: O
Entidad reconstruida (RoBERTa): Estado
Índice: 7, Token: 534, Decoded:  sus, Label: O
Índice: 8, Token: 1495, Decoded:  servicios, Label: O
Índice: 9, Token: 27313, Decoded:  articul, Label: O
Índice: 10, Token: 483, Decoded: ados, Label: O
Índice: 11, Token: 320, Decoded:  a, Label: O
Índice: 12, Token: 365, Decoded:  los, Label: O
Índice: 13, Token: 14266, Decoded:  extremos, Label: O
Índice: 14, Token: 313, Decoded:  de, Label: O
Índice: 15, Token: 332, Decoded:  la, Label: O
Índice: 16, Token: 11550, Decoded:  Amazon, Label: S-LOC
Índice: 17, Token: 404, Decoded: ía, Label: E-LOC
Índice: 18, Token: 379, Decoded:  A, Label: O
Entidad reconstruida (RoBER